In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import math
R = 111194.926644
pi = math.pi
from sklearn.cluster import KMeans

In [178]:
df = pd.read_csv('person.csv')
df.drop(columns='Unnamed: 0', inplace=True)
df_0 = df[df['Floor']==0]
df_1 = df[df['Floor']==1]
df_2 = df[df['Floor']==2]
df_3 = df[df['Floor']==3]

In [252]:
df

,ClientMacAddr,Freq_per_day,lat_std,Stay_duration,lng_std,Category,Floor
0,00:00:00:00:00:00,29.000000,0.018552,2.000000,0.035640,0,0
1,00:0b:82:d0:ff:35,41.294737,0.047863,1.578947,0.047723,0,0
2,00:0c:e7:fe:2a:94,13.000000,0.000000,1.000000,0.000000,0,0
3,00:10:20:ce:df:1d,35.000000,0.037725,3.000000,0.056515,0,0
4,00:b3:62:4e:0f:c9,12.058824,0.100345,1.529412,0.031896,0,0
...,...,...,...,...,...,...,...
11408,f8:a2:d6:b4:66:63,99.000000,0.018395,1.000000,0.038682,0,3
11409,f8:c3:9e:8a:cc:52,161.000000,0.039492,1.000000,0.051798,0,3
11410,f8:e0:79:eb:48:68,18.000000,0.032692,1.000000,0.221697,0,3
11411,f8:ff:c2:c0:0b:8b,237.042553,0.043990,2.326241,0.172328,0,3


In [258]:
df = df.groupby(by='ClientMacAddr', group_keys=False).apply(lambda x: x.sort_values('Floor', ascending=False)).groupby('ClientMacAddr').first().reset_index()

In [274]:
df_m = pd.read_csv('machine.csv')
df_m = df_m.groupby(by='ClientMacAddr', group_keys=False).apply(lambda x: x.sort_values('Floor', ascending=False)).groupby('ClientMacAddr').first().reset_index()

In [276]:
df_m.to_csv('machine_v2.csv')

In [259]:
df.to_csv('person_v2')

最常出现区域在maintenance room网格内的人，认为是maintenance人员

In [261]:
maintenance = pd.read_csv('maintenance.csv')
maintenance

,Unnamed: 0,ClientMacAddr
0,601,80:58:f8:7a:2c:4c
1,2162,40:e2:30:cd:e6:55
2,2301,58:d9:c3:9d:67:e1
3,2380,5c:e0:c5:ad:af:f2
4,2418,60:14:b3:77:98:8b
...,...,...
142,11405,f0:d7:aa:31:73:8b
143,11406,f4:0e:01:08:b0:30
144,11408,f4:7d:ef:19:c1:4a
145,11410,f8:a2:d6:b4:66:63


In [262]:
#no_main: visitor, security, staff
no_main = df[~df['ClientMacAddr'].isin(maintenance.ClientMacAddr)]
no_main

,ClientMacAddr,Freq_per_day,lat_std,Stay_duration,lng_std,Category,Floor
0,00:00:00:00:00:00,29.0,0.018552,2.0,0.035640,0,0
1,00:08:22:24:b7:fb,14.0,0.021206,1.0,0.007572,0,2
2,00:08:22:42:b8:fb,2.0,0.034288,1.0,0.115540,0,2
3,00:08:22:4c:c2:fb,12.0,0.000000,1.0,0.000000,0,2
4,00:08:22:4c:f9:cd,9.0,0.000000,1.0,0.000000,0,1
...,...,...,...,...,...,...,...
10005,fc:77:74:f4:03:94,6.0,0.000000,1.0,0.000000,0,2
10006,fc:a0:1c:cb:9f:6e,14.0,0.022481,1.0,0.047823,0,2
10007,fc:a6:67:be:c0:a5,20.0,0.107590,2.0,0.047680,0,1
10008,fc:aa:b6:e9:40:d4,12.0,0.000000,1.0,0.000000,0,2


**freq_per_day>200, stay_duration>2认为是员工（security+staff）** 

In [264]:
#security, staff
employee = no_main[(no_main.Freq_per_day>200) & (no_main.Stay_duration > 2)]
employee

,ClientMacAddr,Freq_per_day,lat_std,Stay_duration,lng_std,Category,Floor
200,00:23:a7:e5:8c:5c,426.102443,0.034593,2.095585,0.082450,0,2
202,00:23:a7:e7:63:c0,213.117302,0.124944,2.668622,0.033723,0,1
204,00:23:a7:e7:81:3c,342.991361,0.014285,2.062635,0.013274,0,1
222,00:b3:62:25:cf:a2,296.964052,0.066569,3.437908,0.157242,0,3
226,00:b3:62:4e:0f:c9,344.538339,0.072485,3.803514,0.154974,0,3
...,...,...,...,...,...,...,...
9961,f8:38:80:b0:03:6e,603.958959,0.143050,3.039595,0.084159,0,2
9968,f8:63:3f:34:5d:6e,470.287785,0.022534,2.550725,0.428967,0,2
9969,f8:63:3f:43:73:0d,490.133041,0.205207,3.222222,0.062577,0,2
9986,f8:ff:c2:07:a8:3d,583.000000,0.038116,3.000000,0.056306,0,0


In [265]:
no_main['var'] = no_main.lat_std**2 + no_main.lng_std**2

/var/folders/6q/p8cwbrgn2w7314r99d24lh6r0000gn/T/ipykernel_61916/2283344887.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_main['var'] = no_main.lat_std**2 + no_main.lng_std**2


**freq_per_day>200, stay_duration>2， 方差大于0.15 认为是安保人员** 

In [277]:
security = no_main[(no_main.Freq_per_day>250) & (no_main.Stay_duration > 2) & (no_main['var']>0.15)]
security.shape[0]

41

**普通员工** 

In [268]:
#staff id 
staff = employee[~employee['ClientMacAddr'].isin(security.ClientMacAddr)]

,ClientMacAddr,Freq_per_day,lat_std,Stay_duration,lng_std,Category,Floor
200,00:23:a7:e5:8c:5c,426.102443,0.034593,2.095585,0.082450,0,2
202,00:23:a7:e7:63:c0,213.117302,0.124944,2.668622,0.033723,0,1
204,00:23:a7:e7:81:3c,342.991361,0.014285,2.062635,0.013274,0,1
222,00:b3:62:25:cf:a2,296.964052,0.066569,3.437908,0.157242,0,3
226,00:b3:62:4e:0f:c9,344.538339,0.072485,3.803514,0.154974,0,3
...,...,...,...,...,...,...,...
9960,f8:38:80:84:14:61,263.689379,0.057520,2.238477,0.168412,0,2
9961,f8:38:80:b0:03:6e,603.958959,0.143050,3.039595,0.084159,0,2
9969,f8:63:3f:43:73:0d,490.133041,0.205207,3.222222,0.062577,0,2
9986,f8:ff:c2:07:a8:3d,583.000000,0.038116,3.000000,0.056306,0,0


In [269]:
visitor = no_main[~no_main['ClientMacAddr'].isin(employee.ClientMacAddr)]

In [281]:
security['level'] = [3]*security.shape[0]
security.head(5)

/var/folders/6q/p8cwbrgn2w7314r99d24lh6r0000gn/T/ipykernel_61916/2624240367.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  security['level'] = [3]*security.shape[0]


,ClientMacAddr,Freq_per_day,lat_std,Stay_duration,lng_std,Category,Floor,var,level
230,00:b3:62:8e:6b:a7,316.143603,0.262014,2.250000,0.415862,0,2,0.241593,3
418,0c:70:4a:a2:6d:e7,356.570093,0.343856,4.644860,0.321201,0,2,0.221407,3
435,10:02:b5:e3:08:cd,603.947213,0.162533,2.713370,0.389813,0,2,0.178371,3
538,1c:5c:f2:f3:7a:fd,577.278708,0.340275,2.408493,0.212670,0,2,0.161016,3
3006,48:45:20:37:cb:57,495.609615,0.338497,2.909615,0.210882,0,2,0.159052,3


In [283]:
visitor['level'] = [0]*visitor.shape[0]
visitor.head(5)

/var/folders/6q/p8cwbrgn2w7314r99d24lh6r0000gn/T/ipykernel_61916/3073732492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  visitor['level'] = [0]*visitor.shape[0]


,ClientMacAddr,Freq_per_day,lat_std,Stay_duration,lng_std,Category,Floor,var,level
0,00:00:00:00:00:00,29.0,0.018552,2.0,0.035640,0,0,0.001614,0
1,00:08:22:24:b7:fb,14.0,0.021206,1.0,0.007572,0,2,0.000507,0
2,00:08:22:42:b8:fb,2.0,0.034288,1.0,0.115540,0,2,0.014525,0
3,00:08:22:4c:c2:fb,12.0,0.000000,1.0,0.000000,0,2,0.000000,0
4,00:08:22:4c:f9:cd,9.0,0.000000,1.0,0.000000,0,1,0.000000,0


In [284]:
staff['level'] = [1]*staff.shape[0]
staff.head(5)

/var/folders/6q/p8cwbrgn2w7314r99d24lh6r0000gn/T/ipykernel_61916/1267616985.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staff['level'] = [1]*staff.shape[0]


,ClientMacAddr,Freq_per_day,lat_std,Stay_duration,lng_std,Category,Floor,level
200,00:23:a7:e5:8c:5c,426.102443,0.034593,2.095585,0.082450,0,2,1
202,00:23:a7:e7:63:c0,213.117302,0.124944,2.668622,0.033723,0,1,1
204,00:23:a7:e7:81:3c,342.991361,0.014285,2.062635,0.013274,0,1,1
222,00:b3:62:25:cf:a2,296.964052,0.066569,3.437908,0.157242,0,3,1
226,00:b3:62:4e:0f:c9,344.538339,0.072485,3.803514,0.154974,0,3,1


In [287]:
maintenance = df[df['ClientMacAddr'].isin(maintenance.ClientMacAddr)]

In [288]:
maintenance['level'] = [2]*maintenance.shape[0]
maintenance.head(5)

/var/folders/6q/p8cwbrgn2w7314r99d24lh6r0000gn/T/ipykernel_61916/4197533125.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maintenance['level'] = [2]*maintenance.shape[0]


,ClientMacAddr,Freq_per_day,lat_std,Stay_duration,lng_std,Category,Floor,level
293,04:d6:aa:c0:45:a2,13.0,0.000000,1.0,0.000000,0,3,2
295,04:d6:aa:c3:9a:ba,312.0,0.014990,2.0,0.068032,0,3,2
562,24:18:1d:6a:ec:f7,98.0,0.057513,2.0,0.185112,0,3,2
670,24:46:c8:0b:8a:f9,13.0,0.026649,1.0,0.033170,0,3,2
790,24:46:c8:1d:cf:68,13.0,0.000000,1.0,0.000000,0,3,2


In [289]:
security.to_csv('security.csv')
maintenance.to_csv('maintenance.csv')
staff.to_csv('staff.csv')
visitor.to_csv('visitor.csv')

In [290]:
person_level = pd.concat([security, maintenance, staff, visitor])

In [295]:
person_level.to_csv('person_level.csv')